In [1]:
import sys
sys.path.append('scripts')
from lib import *
from sklearn.metrics import confusion_matrix
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go




In [2]:
dfs, available_classes, available_models = read_raw_reports(os.path.join('..',raw_reports_path))

In [3]:
available_classes_r = available_classes[::-1]

In [4]:
available_classes_r

['sett',
 'paving stones',
 'ground',
 'grass',
 'concrete plates',
 'concrete',
 'asphalt']

In [5]:
available_models

['ViT-L_14',
 'RN50x64',
 'ViT-H-14-378-quickgelu',
 'ViT-B_32',
 'EVA02-E-14-plus',
 'ViT-bigG-14-CLIPA-336',
 'RN101',
 'ViT-SO400M-14-SigLIP-384']

In [6]:
available_classes

['asphalt',
 'concrete',
 'concrete plates',
 'grass',
 'ground',
 'paving stones',
 'sett']

In [7]:
all_dfs = []
modelwise_dfs = {}

for class_dict in dfs:
    for model_name in dfs[class_dict]:
        model_df = dfs[class_dict][model_name]
        all_dfs.append(model_df)

        if not model_name in modelwise_dfs:
            modelwise_dfs[model_name] = []

        modelwise_dfs[model_name].append(model_df)


for model_name in modelwise_dfs:
    modelwise_dfs[model_name] = pd.concat(modelwise_dfs[model_name]).copy()

In [8]:
merged_df = pd.concat(all_dfs)

In [9]:
true_classes = merged_df.groupby('image_name').agg('first')[class_label].str.replace('_', ' ')
merged_pred = merged_df.groupby('image_name').sum(numeric_only=True)

merged_pred[prediction_label] = merged_pred.idxmax(axis=1)
merged_pred[class_label] = true_classes


In [10]:
merged_pred

,asphalt,concrete,concrete_plates,grass,ground,paving_stones,sett,prediction,class
image_name,,,,,,,,,
00098db7c6cd6ee36d590ff0d5db7ad1.png,0.340285,4.083931,1.126276,0.129063,0.664580,1.458781,0.197173,concrete,concrete
00157343508e08f1dbc0efc1f8066f3d.png,0.761310,0.329169,0.594061,1.113722,3.021233,1.749222,0.431211,ground,ground
003a08eff81fe589f95102bf2ce1ecf5.png,0.294458,0.142742,0.112069,6.108878,1.087577,0.108795,0.145750,grass,grass
005b94f55127ae460c0741eac3a89b25.png,5.064348,0.371833,1.620817,0.131203,0.337722,0.301726,0.172837,asphalt,asphalt
006d04cdfa79fac8f108a0a1cd34d0f8.png,0.167342,0.174221,0.115470,6.273266,1.022250,0.148719,0.098712,grass,grass
...,...,...,...,...,...,...,...,...,...
ffbd835981460b0e1b7825681c7b9449.png,0.682599,0.679440,0.573372,0.285442,1.097024,4.048435,0.633685,paving_stones,concrete plates
ffd0ab8395fc05d8d14f3cfe8a6a6b4a.png,0.760942,0.671504,0.633202,0.757067,3.890179,0.829439,0.457522,ground,ground
ffd4bc27fe750ca5fedac61da4f7a397.png,0.606997,0.292095,0.913657,0.498596,3.423894,1.748372,0.516377,ground,ground


In [11]:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes_r, normalize='true') * 100
c_mat = c_mat.round(3)

In [12]:
# sanity check on labels:
print(merged_pred[class_label].unique())

['concrete' 'ground' 'grass' 'asphalt' 'sett' 'paving stones'
 'concrete plates']


In [13]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [14]:
n_rows = int(ceil(len(available_models)/2))

In [15]:
modelwise_confusion_matrices = {}

for model in modelwise_dfs:
    true_labels =      modelwise_dfs[model][class_label]#.tolist()
    predicted_labels = modelwise_dfs[model][prediction_label]#.tolist()

    modelwise_confusion_matrices[model] = confusion_matrix(true_labels,predicted_labels,labels=available_classes, normalize='true') * 100
    modelwise_confusion_matrices[model] = modelwise_confusion_matrices[model].round(3)

In [16]:
modelwise_confusion_matrices

{'EVA02-E-14-plus': array([[60.134, 28.508,  0.   ,  5.122,  3.786,  0.   ,  2.45 ],
        [17.277, 73.298,  0.   ,  3.141,  5.497,  0.   ,  0.785],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.439,  0.219,  0.   , 98.465,  0.877,  0.   ,  0.   ],
        [15.171,  1.923,  0.   ,  1.496, 78.419,  0.   ,  2.991],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [19.444, 15.278,  0.   ,  8.333, 56.944,  0.   ,  0.   ]]),
 'RN101': array([[41.86 ,  0.388,  0.   ,  8.14 , 34.109,  0.   , 15.504],
        [17.241,  5.172,  0.   ,  1.724, 37.931,  0.   , 37.931],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   , 85.144, 14.412,  0.   ,  0.443],
        [ 0.539,  0.   ,  0.   ,  1.887, 97.035,  0.   ,  0.539],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [33.333,  0.   ,  0.   ,  8.333, 58.333,  0.   ,  0.   ]]),
 'RN50x64': array([[10.09 , 46.188,  0.   , 

In [17]:
# fig = go.Figure().set_subplots(rows=n_rows, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.05, horizontal_spacing=0.01)
fig = make_subplots(rows=n_rows, cols=2, 
                    shared_xaxes=True, 
                    shared_yaxes=True,
                    subplot_titles=list(modelwise_confusion_matrices.keys()),
                    vertical_spacing=0.015, 
                    horizontal_spacing=0.01,
                    # x_title="Prediction",
                    # y_title="Class",
                    )

for i,matrix in enumerate(modelwise_confusion_matrices):
    subplot = px.imshow(modelwise_confusion_matrices[matrix].T,
                
                x=available_classes,
                y=available_classes,
              text_auto=True, 
              #  aspect="auto",
               )
    
    # print(i//2+1,i%2+1)

    fig.add_trace(subplot.data[0], row=i//2+1, col=i%2+1)
    # fig.append_trace(subplot.data[0], row=i//2+1, col=i%2+1)

fig.update_coloraxes(colorscale=px.colors.sequential.Burg)
fig.update_layout(height=400*n_rows, width=400*2,
                  xaxis_title="Prediction",
                  yaxis_title="Class",
                  showlegend=True,)
fig.show()


In [18]:
# getting the diagonal sum of the confusion matrix:
diagonals = {}

for model in modelwise_confusion_matrices:
    diagonals[model] = modelwise_confusion_matrices[model].diagonal().sum()

diagonals['combined'] = c_mat.diagonal().sum() 

diagonals_df = pd.DataFrame.from_dict(diagonals, orient='index',columns=['score']).reset_index().rename(columns={'index':'model_name'}).sort_values(by='score', ascending=False)

diagonals_df['score'] = round(100 *  diagonals_df['score'] / (len(available_models) * 100),2)

diagonals_df

,model_name,score
5,ViT-H-14-378-quickgelu,47.09
8,combined,41.32
7,ViT-SO400M-14-SigLIP-384,39.06
0,EVA02-E-14-plus,38.79
6,ViT-L_14,32.20
4,ViT-B_32,29.72
1,RN101,28.65
2,RN50x64,23.13
3,ViT-bigG-14-CLIPA-336,8.28


In [19]:
fig = px.bar(diagonals_df,x='model_name', y='score', text_auto=True)
fig.show()


In [20]:
selected_models = {}

for modelname in ['ViT-H-14-378-quickgelu', 'ViT-SO400M-14-SigLIP-384', 'EVA02-E-14-plus']:
    selected_models[modelname] = modelwise_dfs[modelname]

In [21]:
merged_selected = pd.concat(selected_models).reset_index()

In [22]:
true_classes = merged_selected.groupby('image_name').agg('first')[class_label].str.replace('_', ' ') #.reset_index().rename(columns={class_label:'class'})

In [23]:
merged_selected = merged_selected.groupby('image_name').sum(numeric_only=True)

In [24]:
merged_selected.drop(columns=['level_1'], inplace=True)

In [25]:
merged_pred[prediction_label] = merged_selected.idxmax(axis=1)
merged_pred[class_label] = true_classes.tolist()

In [26]:
# computing the confusion matrix:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes_r, normalize='true') * 100
c_mat = c_mat.round(3)

In [27]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [28]:
100 * (c_mat.diagonal().sum() / (100 * len(available_models)))

43.315999999999995